# Project 5 -- TrackPy -- MCEN 1030

Modified from the TrackPy walthrough found here: http://soft-matter.github.io/trackpy/dev/tutorial/walkthrough.html




## Imports

In [ ]:
# run these first three cells once, they take some time!

from __future__ import division, unicode_literals, print_function  # for compatibility with Python 2 and 3

import matplotlib as mpl
import matplotlib.pyplot as plt

%matplotlib inline

mpl.rc('figure',  figsize=(10, 5))
mpl.rc('image', cmap='gray')

In [ ]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series  # for convenience

!pip install pims
!pip install trackpy
import pims
import trackpy as tp

In [ ]:
@pims.pipeline
def gray(image):
    return image[:, :, 1]  # Take just the green channel

frames = gray(pims.open('/content/snow2.mp4')) # the video I took last night

## "Frames"

In addition to trackpy, we have also imported "pims": Python IMage Sequence. With thispackage we are able to create an array of images, and can access, say, the zeroth element of the array with frames[0].

In [ ]:
frames # will tell you some technical detail about the frames "array", including the size

In [ ]:
plt.imshow(frames[0]); # can see the zeroth element of the array, an image

We will discuss the image size in pixels later (i.e., "'x' size" above) and the fact that the origin in this figure is in the top-left of the image, with y increasing as you go down.

## Identifying "features"

We will examine just the zeroth frame, at first, and use the command "locate" to find "features" -- bright spots that we will track -- within the frame. The idea is to pick decent values for the parameters for this one frame, and then hopefully all the frames will be able to be processed with similar values.

In the "locate" command below, the numerical value is an estimate of the diameter of the feature, in pixels. From above, you know the size of the image in the x and y directions. So, how big are the snowflakes?

The "annotate" command takes the locations, stored in f, and plots them onto the frame, so you'll be able to see how it's looking. Don't try to be perfect... after all, the snowflakes are different size/different distances away. Err on the side of getting too many, and we'll refine later. It will be something like 150-200.



In [ ]:
f = tp.locate(frames[0], 3) # 3 is the diameter in pixels, choose something bigger than 3.

tp.annotate(f, frames[0]);

Here is the data you/TrackPy produced, just for the first frame:

In [ ]:
f.head()  # shows the first few rows of data

## Refine the list

One way to refine this list a bit is to look at the "mass" (basically the total brightness) of each feature. In our case I like the interpretation: the bright features are the ones that are going to be easiest to track, and so let's focus on them.

Matplotlib has a way to plot the count vs mass, a histogram:

In [ ]:
fig, ax = plt.subplots()
ax.hist(f['mass'], bins=100)
# we imported another new package, pandas.
# Pandas lets you access data like it's a dictionary, here with key 'mass'.

ax.set(xlabel='mass', ylabel='count');

What is the minimum mass among the features you identified? (It will depend on your choice of diameter.)

In the following, we will recreate the list using the "minmass" command... choose something bigger, to ignore the fainter objects. Not too big, you should still see 200 or so particles (and will also probably get the lamp post quite a few times).

In [ ]:
f = tp.locate(frames[0], 3, minmass=1) # adjust with your values of diameter and minmass
tp.annotate(f, frames[0]);

## Locate features in all the frames

Now is where it gets interesting: "batch" is basically "locate" on the whole frame. If we include the diameter value and minmass value, we will look for snowflakes that match that criterion in every frame.

In [ ]:
f = tp.batch(frames, 3, minmass=1, processes=1);
# adjust with your diameter and minmass picks
# Keep processes=1, else Colab crashes
# (Otherwise it tries to do a parallel calculation and can't, I think)

... and then "link" will attempt to connect the snowflake locations based on a search size range: 5 is probably OK.

In [ ]:
t = tp.link(f, 5, memory=3)

## Pruning the data a bit more

We can maybe clean up the data set a bit more, removing information that is not going to help in our quest.

One idea: Let's take a look at the feature size as a function of mass, to see if there is anything weird:

In [ ]:
plt.figure()
tp.mass_size(t.groupby('particle').mean());

In [ ]:
t2 = t[(t['size'] < 500.) & (t['ecc'] < 0.3)]
# I notice a weird horizontal line somewhere, not at 500. Replace that value with something useful.
# And also the "eccentricity" is a measure of how circular something is, with ecc=0 being a circle.
# The second part of this removes all features that are weirdly shaped, keep it.

Compare the next two cells: before and after this pruning step

In [ ]:
plt.figure()
tp.plot_traj(t);

In [ ]:
plt.figure()
tp.plot_traj(t2);

In the "before", you probably see a few almost-vertical lines on the right side of the image. What are they? And hopefully they are gone in the second.

## Determining the snowflake speed

There is a built-in command! The following calculates the average displacement of the particles (i.e., how much did they drift?) as a function of frame (i.e., time):

In [ ]:
d = tp.compute_drift(t2)
d.plot()
plt.show()

You should see a positive displacement in the y-direction and a negative displacement in the x-direction. Why? Hint: Look back at plt.imshow(frames[0]);

---



With this, we are able to determine the physical speed of the snowflakes. Here are some numbers, see if you can figure it out:

I estimate the diameter of the lamp post is 25 cm, and early we described the width of the image in pixels. How many cm per pixel?

The framerate of my camera was 24 frames per second.

How can we get a speed, in cm/s, from this information?

And when you get your answer, is it reasonable?